In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
from collections import OrderedDict, namedtuple
from sqlalchemy import func, distinct, text, and_
import pandas as pd
from IPython.display import display, HTML, Markdown
import seaborn as sns

import util
from util.year import year as current_year
from db import session
import model
from datetime import datetime
import json

pd.options.display.float_format = '{:.2f}'.format
plt.rcParams['figure.figsize'] = [10, 8]
print(datetime.now())

2019-05-12 14:40:02.942354


In [22]:
all_waves = session.query(model.Wave).\
    filter(model.Wave.year == current_year.id).\
    order_by(model.Wave.index).all()

In [36]:
Feedback = namedtuple(
    'Feedback',
    ['user', 'task', 'explained', 'interesting', 'difficult']
)

feedbacks_raw = session.query(
    model.Feedback,
    model.Task,
).\
    join(model.Feedback.r_task).\
    join(model.Task.r_wave).filter(model.Wave.year == current_year.id).\
    join(model.Feedback.r_user).\
    filter(model.User.role == 'participant').\
    order_by(model.Task.id).all()

feedbacks = []
for f, task in feedbacks_raw:
    data = {
        item['id']: item['answer']
        for item in json.loads(f.content)
    }
    feedbacks.append(Feedback(
        f.user,
        task,
        data['explained'] if 'explained' in data else None,
        data['interesting'] if 'interesting' in data else None,
        data['difficult'] if 'difficult' in data else None,
    ))

In [37]:
large_tasks = util.task.large_tasks().all()

In [41]:
def show_wave_feedbacks(wave):
    feedbacks_in_wave = [ 
        f
        for f in feedbacks
        if f.task.wave == wave.id
    ]
    feedbacks_in_wave.sort(
        key=lambda x: (x.task in large_tasks, x.task.id)
    )
    
    df = pd.DataFrame(feedbacks_in_wave)
    grouped = df.groupby('task')
    per_task = pd.DataFrame(OrderedDict((
        ('Explained Count', grouped.explained.count()),
        ('Interesting Count', grouped.interesting.count()),
        ('Difficult Count', grouped.difficult.count()),
        ('Explained Mean', grouped.explained.mean()),    
        ('Interesting Mean', grouped.interesting.mean()),    
        ('Difficult Mean', grouped.difficult.mean()),
    )))
    
    s = per_task.style

    s.format({
        'Explained Mean': '{:.2f}'.format,
        'Interesting Mean': '{:.2f}'.format,
        'Difficult Mean': '{:.2f}'.format,
    })
    
    s.bar(subset=[
        'Explained Count', 'Interesting Count', 'Difficult Count'
    ], color='#5fd65f')
    
    s.background_gradient(subset=[
        'Explained Mean',
        'Interesting Mean',
    ], cmap=sns.light_palette("green", as_cmap=True))
    
    s.background_gradient(subset=[
        'Difficult Mean',
    ], cmap=sns.light_palette("red", as_cmap=True))    
    
    display(Markdown('## {name}'.format(name=wave.caption)))
    display(s)

for wave in all_waves:
    show_wave_feedbacks(wave)

## Nultá vlna

,Explained Count,Interesting Count,Difficult Count,Explained Mean,Interesting Mean,Difficult Mean
task,,,,,,
Vítej v KSI,68,68,68,4.07,3.26,1.56
Vaření proměnných,70,70,70,3.87,3.50,1.84
Seznámení s pythonem,64,64,64,4.11,3.69,1.98
Složitost,51,51,51,4.10,3.61,1.61
Korektnost,40,40,40,4.08,3.90,1.80
Úvod do seznamů,35,35,35,3.46,3.69,2.43
Funkce,37,37,37,3.95,3.62,2.05
Start pro zkušené,30,30,30,3.93,3.70,2.20
Asymptotická složitost,39,39,39,2.92,3.00,2.82


## Hrubá síla

,Explained Count,Interesting Count,Difficult Count,Explained Mean,Interesting Mean,Difficult Mean
task,,,,,,
Rekurze,31,31,31,3.90,3.45,1.71
Brute-force,43,43,43,4.21,3.49,1.44
Permutace,37,37,37,3.35,3.70,2.38
Variace s opakováním,32,32,32,3.66,3.34,1.72
Variace bez opakování,28,28,28,4.11,3.96,3.00
Generování podmnožin,27,27,27,4.04,3.96,3.15
Backtracking,27,27,27,3.89,4.00,2.78
Problém osmi dam,18,18,18,4.72,4.67,3.61
Jigsaw Sudoku,7,7,7,4.71,4.57,3.71


## Rozděl a panuj

,Explained Count,Interesting Count,Difficult Count,Explained Mean,Interesting Mean,Difficult Mean
task,,,,,,
Zmenši a panuj,22,22,22,4.05,3.23,1.27
Transformuj a panuj,14,14,14,3.50,3.93,1.79
Rozděl a panuj,13,13,13,3.85,3.62,1.85
Binární vyhledávání,13,13,13,4.31,4.23,2.31
Rozděluj a slučuj,11,11,11,3.45,3.91,3.36
QuickSort,7,7,7,4.14,4.00,2.71
Rekurzivní volání,16,16,16,2.94,2.94,1.38
Binárne vyhľadávanie 2.0,12,12,12,4.33,4.00,3.08
Karlík jede na soustředění,7,7,7,4.00,3.43,2.43


## Dynamické programování

,Explained Count,Interesting Count,Difficult Count,Explained Mean,Interesting Mean,Difficult Mean
task,,,,,,
Úvod do dynamického programování,18,18,18,3.94,3.50,1.61
Překrývající se problémy,13,13,13,2.54,2.62,2.31
Bottom-up,12,12,12,3.67,3.58,2.50
Rekurentné vzťahy,13,13,13,3.54,3.08,2.08
Rekonstrukce řešení,13,13,13,2.31,3.23,2.23
Nejdelší rostoucí podposloupnost,5,5,5,3.80,3.40,2.60
Batoh,17,17,17,3.59,3.71,2.71
Mince,14,14,14,3.79,4.14,2.64
LosKarlosovské součty,6,6,6,4.83,3.83,3.67


## NP-těžká vlna

,Explained Count,Interesting Count,Difficult Count,Explained Mean,Interesting Mean,Difficult Mean
task,,,,,,
Opakování složitosti,9,9,9,3.67,3.11,1.00
Složitostní třídy,8,8,8,4.00,3.50,1.12
NP aneb ověřování řešení,4,4,4,5.00,5.00,2.25
Redukce,6,6,6,3.33,4.17,2.17
Trénink redukcí,7,7,7,4.86,4.57,1.29
O NP-úplných problémoch,7,7,7,4.43,4.14,2.00
NP hra a řešení složitých problémů,7,7,7,4.86,4.43,1.57
Strategie pro NP hru,3,3,3,5.00,5.00,3.67
Tvorba karty,5,5,5,5.00,4.40,2.60
